# **机器学习新年公开课 - 模型融合实战技巧精讲**
## Day 1 融合入门：Voting与Averaging
> 节选自《2022机器学习实战》正课<br>
> 作者：@菜菜TsaiTsai<br>
> 版本号：2022/1/16 V3<br>

## 课程规划

欢迎来到为期三天的《制霸竞赛：融合模型实战技巧精讲》公开课。在这门课程中，我将用三天时间带你了解模型融合中的四大核心方法：Averaging，Voting，Stacking与Blending，我们会讲解每种方法的基本流程、提供完成融合的基础代码、同时深入探讨每种方法在实际应用中的所涉及到的核心技巧以及理论局限性。**当你完成这门课程时，你将能够自由地使用以上任意一种方法完成基本的融合**。如果你想更深入地学习更多模型融合相关的知识、包括如何在竞赛中使用融合提分、融合应用的相关技巧等，则可以向客服小可爱咨询正课《2022机器学习实战》。

**DAY 1：融合入门：Voting与Averaging**

**DAY 2：融合进阶：堆叠法Stacking与Blending**

**DAY 3：多模型Stacking的综合应用实战**

为掌握这门课程，你需要先掌握：
- Python基础 >> 《九天老师的Python基础》<br>https://www.bilibili.com/video/BV1U54y1W7jw?spm_id_from=333.999.0.0

- 机器学习基础 >> 《2022机器学习实战公开课》<br>包括超参数、泛化能力等基本概念<br>交叉验证等基本工具<br>随机森林、GBDT等集成算法<br>https://www.bilibili.com/video/BV1f3411r7EP?share_source=copy_web

In [1]:
#版本确认
import re
import numpy as np
import pandas as pd
import matplotlib as mlp
import matplotlib.pyplot as plt
import sklearn
import xgboost as xgb

In [2]:
for package in [sklearn,mlp,np,pd,xgb]:
    print(re.findall("([^']*)",str(package))[2],package.__version__)

sklearn 1.0.1
matplotlib 3.4.3
numpy 1.21.4
pandas 1.3.4
xgboost 1.5.1


In [ ]:
#pip install --upgrade scikit-learn
#conda update scikit-learn

#使用清华源配置xgboost
#!pip install xgboost -i https://pypi.tuna.tsinghua.edu.cn/simple

**目录**

一 认识模型融合<br>
二 投票法Voting<br>
&emsp; 1 五大类投票方法<br>
&emsp; 2 使用sklearn实现投票法<br>
&emsp;&emsp; 2.1 导入所需的工具库，确认数据<br>
&emsp;&emsp; 2.2 定义交叉验证所需要的函数<br>
&emsp;&emsp; 2.3 建立基于交叉验证的benchmark、做模型选择<br>
&emsp;&emsp; 2.4 构建多组分类器，进行融合<br>
&emsp;&emsp; 2.5 构建多样性<br>
&emsp;&emsp;&emsp; 2.5.1 多种多样性混合<br>
&emsp;&emsp;&emsp; 2.5.2 剔除不良算法<br>
&emsp;&emsp;&emsp; 2.5.3 精简多样性<br>
&emsp;&emsp; 2.6 分类器加权<br>

# 一 认识模型融合

在机器学习竞赛界，流传着一句话：**当一切都无效的时候，选择模型融合**。这句话出自一位史上最年轻的Kaggle Master之口，无疑是彰显了模型融合这一技巧在整个机器学习世界的地位。如果说机器学习是人工智能技术中的王后，集成学习(ensemble Learning)就是王后的王冠，而坐落于集成学习三大研究领域之首的模型融合，则毫无疑问就是皇冠上的明珠，熠熠生辉，夺人眼球。

为什么模型融合如此受学者们青睐呢？模型融合在最初的时候被称为“评估器结合”，与集成算法(狭义)一样，它也是训练多个评估器、并将多个评估器以某种方式结合起来解决问题的机器学习方法。我们都知道，集成算法(狭义)是竞赛高分榜统治者，但在高分榜上最顶尖的那部分战队一定都使用了相当丰富的模型融合技巧，**因为模型融合能够在经典集成模型的基础上进一步提升分数，使用模型融合技巧后、融合算法的效果常常可以胜过当前最先进的单一集成算法**。正是这一有效的提分能力，让模型融合在竞赛界具有很高的地位。也因此，模型融合是提分的最终手段。

同样是使用多个评估器的技术，模型融合为什么能比集成算法获得更高的分数呢？上世纪90年代的多篇论文(*Xu et al., 1992; Bauer & Kohavi, 1999; Optiz & Maclin, 1999*)从多个角度给出了多个理由——

- **直观印象：训练的基础不同**
> **集成算法(狭义)：青铜算法+人海战术**<br>
> - 以Bagging和Boosting代表的集成算法(狭义)可以说是使用青铜算法+人海战术。在集成时，我们一般使用数百棵决策树、甚至树桩这样的弱评估器，构成强大的学习能力。<br><br>
> - 在这个过程中，由于评估器数量很多，因此每个评估器贡献自己的一小步(Boosting)、或者一小点意见(Bagging)，没有任何弱评估器可以在数据集上直接给出优秀的结果。因此从本质上来说，集成算法还是一个算法，弱评估器本身不具备有效预测能力。<br><br>
> **模型融合：王者算法+精英敢死队**<br>
> - 模型融合则使用王者算法+精英敢死队战术。在融合时，我们往往构建最多数十个Xgboost、神经网络这样的强评估器，再设计出强大的规则来融合强评估器的结果。或者，我们使用少数几个很强的学习器处理特征，再让将新特征输入相对较弱的学习器，以此来提升模型表现。<br><br>
> - 在这个过程中，由于评估器很少，所以每个评估器的贡献都很大，即便不融合，单一评估器也可以直接在数据集上给出优秀的结果。因此从本质上来说，模型融合是囊括多个算法的技巧，而并非一个算法。<br>

不难发现：<font color="red">**通过集成，我们让算法从弱到强，但通过融合，我们让算法从强到更强**</font>。面对同样的预测任务，集成算法会努力获得一个好结果，而模型融合则是将许多好结果融合在一起，得到更好的结果。

- **统计方面：假设空间不同，泛化能力不同**
> **集成算法(狭义)：一维假设空间**<br>
> - 每个算法都是一套独一无二的、从数据到结论的计算流程，这一流程在统计学上被称为是“对样本与标签关系的假设”。例如，KNN假设，相同类别的样本在特征上应该相似。Boosting算法假设令单一损失函数最小的树一定可以组成令整体损失最小的树林。<br><br>
> - 我们可以有不同的假设，但在有限的训练数据集上，我们很难看出哪个假设才是对贴近于真实样本标签关系的假设。这就是说，**我们无法保证现在表现好的算法在未知数据集上表现也一定会好**。因此使用单一假设/单一算法的缺点就很明显：虽然在当前数据集上当前假设表现最好，但如果其他假设才更贴近数据真实的情况，那当前选择出的模型的泛化能力在未来就无法得到保证了。<br><br>
> **模型融合：高维假设空间**<br>
> - 相对的，如果能够将不同的假设结合起来，就可以降低选错假设的风险。最贴近数据真实状况的假设，即便在当前数据集上表现并不是最好的，但在融合之后可以给与融合的结果更强大的泛化能力。<br><br>
> - 同时，假设真正的数据规律并不能被现在的数据代表，那融合多个假设也能够拓展假设空间，不同假设最终能够更接近真实规律的可能性也越大。

因此，从统计学的角度来说，模型融合能够：<font color="red">**(1) 降低选错假设导致的风险、(2)提升捕捉到真正数据规律的可能性、(3)提升具有更好泛化能力的可能性**</font>。

- **计算方面：起点数量不同，模型表现不同**
> **集成算法(狭义)：只有一组起点，最终只能得到一个局部最小值**<br>
> - 许多算法在搜索时都会陷入局部最优解，即便有足够多的训练数据，我们也无法判断能否找到理论上真实的最小值。一般在算法训练时，我们把搜索出的局部最小值当作是真实最小值的近似值，但我们却无法估计真实最小值与局部最小值之间有大多的差异。但由于我们只有一个局部最小值，真实最小值在哪里完全没有头绪。<br><br>
> **模型融合：多组不同的起点，最终得到一个局部最小值的范围**<br>
> - 模型融合过程中，每个算法都会有自己的起点、并最终获得多个不同的局部最小值。虽然这些局部最小值当中，没有任意一个等于真正的最小值，但真正的最小值很大可能就在这些局部最小值构成的范围当中。通过结合局部最小值，可以获得更接近真实最小值的近似值，从而提升模型的精度。

因此，从计算方面来看，模型融合能够：<font color="red">**(1)降低选错起点假设的风险、(2)降低局部最小值远远偏离真实值的风险、(3)提升算法结果更接近真实最小值的可能性**</font>。

当算法出现统计学方面的问题时，模型一般表示为高方差、低偏差（更换数据之后模型表现非常动荡）。当算法出现计算方面的问题时，模型一般表示为高偏差、低方差（无论换什么数据，局部最小值都远离真正的最小值）。因此，理论上来说，模型融合这一技巧的上限非常高：我们可以同时降低方差和偏差、找到方差-偏差均衡的最优点。

现在，使用比较广泛的融合方式有如下4种：

- **均值法Averaging**：适用于回归类算法，将每个评估器的输出做平均，类似于Bagging中回归的做法。我们已经在之前的《集成算法》公开课中给大家讲解过Averaging。
- **投票法Voting**：适用于分类算法，按每个评估器的输出进行投票，类似于Bagging中分类的做法。

- **堆叠法Stacking**：使用一个/多个算法在训练集上输出的某种结果作为下一个算法的训练数据。其中比较著名的是gbdt+lr，其中逻辑回归lr使用gbdt输出的树结构数据作为训练数据。

- **改进堆叠法Blending**：一种特殊的stacking，使用一个/多个算法在验证集上输出的某种结果作为下一个算法的训练数据。

另外还有其他在论文中出现过，但并未被广泛使用的各种融合方法，例如无限集成（Unlimited Ensemble）、代数集成（Algebraic Method）、行为知识空间法（behavior knowldge space）等。这些方法都基于非常严谨的数学过程建立，数学功底不错的小伙伴可以查看周志华教授的《集成学习：基础与算法》书籍。接下来的3天，我们将一一说明讲解上述融合方法。

# 二 投票法Voting

## 1 五大类投票方法

投票法是适用于分类任务的模型融合方法，也是在直觉上来说最容易理解的方法。简单来说，**投票法对所有评估器输出的结果按类别进行计数，出现次数最多的类别就是融合模型输出的类别**。假设现在面临3分类问题，有5个强学习器，对于任意样本$i$而言，这些强学习器的输出类别分别如下所示：

1号分类器：3<br>
2号分类器：3<br>
3号分类器：2<br>
4号分类器：3<br>
5号分类器：2<br>

在5个学习器的结果来看，类别3出现3次，类别2出现2次，类别1一次也没有出现，因此出现次数最多的类别3就是该融合模型在样本$i$上的预测类别。这种方式其实就是让所有评估器对类别进行投票、并且让少数如从多数，我们在KNN以及Bagging算法（如随机森林分类）当中都见过这种投票方式，应该相当容易理解。在实际机器学习执行过程当中，我们可以使用多种、丰富的投票方式：

- **相对多数投票 vs 绝对多数投票**
> **相对多数投票**就是上面所提到的少数服从多数，即只要有一个备选类别占比较多即可。与之相对的是绝对多数投票：<br><br>
> **绝对多数投票**要求至少有50%或以上的分类器都输出了同一类别，否则就拒绝预测。例如在上面5个分类器中，至少要有2.5个分类器都输出同一类别，样本才能够进行输出具体类别，否则则输出“不确定”，相当于增加一类输出类别。<br><br>
这种投票方法在现在听起来非常奇怪，为什么我们要允许算法拒绝预测呢？事实上，绝对多数投票是现实中非常常见的投票方法（例如公司关键议程的通过、某些公司董事长的选举），这种投票方法不会轻易抹杀少数派的意见，而会尽量为更多的人争取利益。因此，当我们需要对决策更加谨慎时，我们就会选择绝对多数投票。如果票数最多的类别没有占比超过50%，则重新投票。<br><br>
在机器学习当中，**绝对多数投票可以帮助我们衡量当前投票的置信程度**。票数最多的类别占比越高，说明融合模型对当前样本的预测越有信心。当票数最多的类别占比不足50%时，重新投票对于机器学习来说效率太低，因此我们往往会选择另一种方案：我们可以使用相对较弱的学习器进行绝对多数投票融合、并将绝对多数投票中被输出为“不确定”的样本交由学习能力更强的模型进行预测，相当于将“困难样本”交由复杂度更高的算法进行预测，这也是一种融合方式。

- **硬投票 vs 软投票**
> 大多数分类模型的模型输出结果可以有两种表示方式：<br>
> - 表示为二值或多值标记，即算法输出值为[0,1]或[1,2,3]这样的具体类别。<br><br>
> - 表示为类别概率，即算法输出值为(0,1)之间的任意浮点数，越靠近1则说明算法对当前类别预测的置信度越高。我们需要规定阈值、或使用softmax函数的规则来将浮点数转换为具体的类别。<br><br>
>
> 基于此，我们可以将投票分成两种类型：<br><br>
> **硬投票**：将不同类别的出现次数进行计数，出现次数最多的类别就是投票结果<br><br>
> **软投票**：将不同类别下的概率进行加和，概率最高的类别就是投票结果。基于概率的软投票可以衡量投票的置信程度，某一类别下的概率越高，说明模型对该类别的预测结果越有信心。<br>

具体来看，延续3分类、5个分类器的假设，当使用二值标记时我们有：

1号分类器：3<br>
2号分类器：3<br>
3号分类器：2<br>
4号分类器：3<br>
5号分类器：2<br>

当使用类别概率时，我们有：

|分类器|1的概率|2的概率|3的概率|
|:-:|:-:|:-:|:-:|
|1号|0.1|0.4|0.5|
|2号|0.2|0.35|0.45|
|3号|0.1|0.6|0.3|
|4号|0.3|0.3|0.4|
|5号|0.4|0.5|0.1|

在软投票中，我们对所有分类器上每一个类别的概率进行加和：

|分类器|1的概率|2的概率|3的概率|
|:-:|:-:|:-:|:-:|
|1号|0.1|0.4|0.5|
|2号|0.2|0.35|0.45|
|3号|0.1|0.6|0.3|
|4号|0.3|0.3|0.4|
|5号|0.2|0.7|0.1|
|**加和**|**0.09**|**2.35**|**1.75**|

加和之后，概率占比最高的是类别2。因此基于软投票，算法输出的类别为类别2。不难发现，软投票和硬投票得出的结果可能不同，这是因为软投票有一种“窥探心灵”的能力（也就是衡量置信度的能力）。

如果使用硬头票，那毫无疑问输出类别应该是3。但当我们查看具体的预测概率时我们就会发现，给与类别3较高概率的分类器（1号、2号与4号）往往也给与了类别2较高的概率，只不过这些分类器稍稍倾向于类别3。然而，给出类别2的分类器（3号、5号）往往会给与类别2非常高的概率，这说明这两个分类器对于自己输出的类别非常有信心。这种情况导致对概率进行加和后，我们发现类别2的概率是最高的。在5个分类器进行投票时，有三个分类器认为类别2是真实标签的可能性很高，另外两个分类器认为类别2就是真实标签，那毫无疑问选择输出类别2是整个团队置信度到达顶峰的选项。

<font color="red">**在实际进行投票法融合时，我们往往优先考虑软投票 + 相对多数投票方案**</font>，因为软投票方案更容易在当前数据集上获得好的分数。然而，如果我们融合的算法都是精心调参过的算法，那软投票方案可能导致过拟合。因此具体在使用时，需要依情况而定。

- **加权投票**
> 除了基于概率/基于类别进行投票、以及使用绝对多数/相对多数进行投票之外，还有一种可以与上述方式结合的投票方法，即加权投票。加权投票是在投票过程中赋予不同分类器权重的投票，具体表现在：
> - 对于硬投票，加权投票可以改变每一个分类器所拥有的票数。<br><br>
> - 对于软投票，加权投票可以改变每一个分类器所占的权重，令概率的普通加和变成加权求和。

具体来看，硬投票情况下，我们有：

|分类器|输出类别|原始票数|**权重**|**加权票数**|
|:-:|:-:|:-:|:-:|:-:|
|1|3|1|0.3|0.3|
|2|3|1|0.8|0.8|
|3|2|1|0.5|0.5|
|4|3|1|0.6|0.6|
|5|2|1|0.2|0.2|

现在，我们再按照类别进行计数，类别3的票数为(0.3 + 0.8 + 0.6 = 1.7票)，而类别2的票数为(0.5 + 0.2 = 0.7)票，因此票数占比更多的是类别3，融合模型输出类别3。

在软投票情况下，我们有：

|分类器|1的概率|2的概率|3的概率|**权重**|
|:-:|:-:|:-:|:-:|:-:|
|1号|0.1|0.4|0.5|0.3|
|2号|0.2|0.35|0.45|0.8|
|3号|0.1|0.6|0.3|0.5|
|4号|0.3|0.3|0.4|0.6|
|5号|0.2|0.7|0.1|0.2|

则加权求和结果可以计算为：

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021MachineLearning/Ensembles/Fusion/03.png)

不难发现，当调整权重之后，最终输出的结果依然是类别2，但是加权之后整体算法对于类别2的置信度是降低的、对于类别3的置信度是提升的。因此，修改权重很可能会影响融合模型最终的输出。<font color="Red">**通常来说，我们会给与表现更好的分类器更多的权重，但并不会给与它过多的权重，否则会过拟合**</font>。

大多数时候，我们可能会主观地认为，加权后的模型比未加权的模型更强大，因为加权求和比普通求和更复杂、加权平均也比普通平均更复杂。但事实上，无论是从实验还是理论角度，加权方法并没有表现出比一般普通方法更强的泛化能力。在某些数据上，加权会更有效，但如果强评估器之间的结果差异本来就不大，那加权一般不会使模型结果得到质的提升。

## 2 **使用sklearn实现投票法**

在sklearn的集成学习板块中，拥有数个帮助我们实现各类融合方法的类，其中，我们可以很容易使用下面的类来实现投票法：

*class* `sklearn.ensemble.VotingClassifier`(estimators, *, voting='hard', weights=None, n_jobs=None, flatten_transform=True, verbose=False)<br>

|参数|说明|
|:-:|-|
|estimators|需要使用投票法融合的分类评估器及分类评估器的名称。<br>可输入单一评估器，或使用列表打包多个评估器。|
|voting|投票方式，默认使用相对多数投票方法。<br>输入字符串'hard'，选择硬投票，输入字符串'soft'，选择软投票。当选择软投票方法时，只能接受可以输出概率值的算法，如SVM等间隔度量算法则不在可以融合的范围之内。|
|weights|权重，默认为None，表示不对评估器进行加权。如果输入权重，则使用列表打包多个权重。|
|flatten_transform|当使用软投票时，可以通过该参数选择输出的概率结构。如果为True，最终则输出结构为(n_samples, n_estimators * n_classes)的二维数组。如果为False，最终则输出结构为(n_samples,n_estimators,n_classes)的三维数组。|
|n_jobs, verbose|线程数与模型监控|

### **1 导入所需的工具库，确认数据**

In [107]:
#常用工具库
import re
import numpy as np
import pandas as pd
import matplotlib as mlp
import matplotlib.pyplot as plt
import time

#算法辅助 & 数据
import sklearn
from sklearn.model_selection import KFold, cross_validate
from sklearn.datasets import load_digits #分类 - 手写数字数据集
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

#算法（单一学习器）
from sklearn.neighbors import KNeighborsClassifier as KNNC
from sklearn.neighbors import KNeighborsRegressor as KNNR
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import LogisticRegression as LogiR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import GradientBoostingRegressor as GBR
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb

#融合模型
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import VotingRegressor

在正式课程中我们将使用Kaggle数据集进行集成，在直播课程当中为了运行快速我们使用sklearn自带的手写数字数据集：

In [108]:
data = load_digits()
X = data.data
y = data.target

In [109]:
X.shape

(1797, 64)

In [110]:
np.unique(y) #10分类

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [111]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,y,test_size=0.2,random_state=1412)

我们将在Xtrain，Ytrain上进行交叉验证，并在Xtest，Ytest上进行最后的测试。**我们的目标是：交叉验证的结果尽量好（理论泛化能力强），同时测试集上的结果也需要尽量好（理论泛化能力的验证，但不完全严谨）。当两者不可兼得时，我们优先考虑交叉验证的结果（理论泛化能力强）**。

### **2 定义交叉验证所需要的函数**

In [112]:
def individual_estimators(estimators):
    """
    对模型融合中每个评估器做交叉验证，对单一评估器的表现进行评估
    """
    for estimator in estimators:
        cv = KFold(n_splits=5,shuffle=True,random_state=1412)
        results = cross_validate(estimator[1],Xtrain,Ytrain
                             ,cv = cv
                             ,scoring = "accuracy"
                             ,n_jobs = -1
                             ,return_train_score = True
                             ,verbose=False)
        test = estimator[1].fit(Xtrain,Ytrain).score(Xtest,Ytest)
        print(estimator[0]
          ,"\n train_score:{}".format(results["train_score"].mean())
          ,"\n cv_mean:{}".format(results["test_score"].mean())
          ,"\n test_score:{}".format(test)
          ,"\n")

In [113]:
def fusion_estimators(clf):
    """
    对融合模型做交叉验证，对融合模型的表现进行评估
    """
    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    results = cross_validate(clf,Xtrain,Ytrain
                             ,cv = cv
                             ,scoring = "accuracy"
                             ,n_jobs = -1
                             ,return_train_score = True
                             ,verbose=False)
    test = clf.fit(Xtrain,Ytrain).score(Xtest,Ytest)
    print("train_score:{}".format(results["train_score"].mean())
          ,"\n cv_mean:{}".format(results["test_score"].mean())
          ,"\n test_score:{}".format(test)
         )

### **3 建立基于交叉验证的benchmark、做模型选择**

一般在模型融合之前，我们需要将所有可能的算法都先简单运行一次，然后从中选出表现较好的算法作为融合的基础。同时，我们还可能在分数最高的单一算法上进行精确的调优，找到单一算法可以实现的最好分数来作为benchmark。毕竟，融合要求使用多个算法、自然也会在运算时间和算力上有所要求，如果单一算法的结果能够胜过融合，那我们优秀选择单一算法。在公开课当中，鉴于有限的时间，我为大家挑选了几个算法用于展示投票法融合的实力。我们使用备选分类器中的逻辑回归作为benchmark。

In [327]:
logi = LogiR(max_iter=3000, n_jobs=8) #初始情况下给与一个较大的max_iter，方便迭代到收敛

In [328]:
fusion_estimators(logi) #正处于过拟合状态，需要调整

train_score:1.0 
 cv_mean:0.9666061749903212 
 test_score:0.9527777777777777


### **4 构建多组分类器，进行融合**

- **第一组分类器**：放任自由，收敛为主，有较高过拟合风险

In [346]:
clf1 = LogiR(max_iter = 3000,random_state=1412,n_jobs=8)
clf2 = RFC(n_estimators= 100,random_state=1412,n_jobs=8)
clf3 = GBC(n_estimators= 100,random_state=1412)

estimators = [("Logistic Regression",clf1), ("RandomForest", clf2), ("GBDT",clf3)]
clf = VotingClassifier(estimators,voting="soft")

In [330]:
individual_estimators(estimators)

Logistic Regression 
 train_score:1.0 
 cv_mean:0.9666061749903212 
 test_score:0.9527777777777777 

RandomForest 
 train_score:1.0 
 cv_mean:0.9763574332171892 
 test_score:0.9833333333333333 

GBDT 
 train_score:1.0 
 cv_mean:0.9686943476577623 
 test_score:0.9388888888888889 



不难发现，每个模型在训练集上的分数都达到了1，但交叉验证分数大约在96%~97%之间，与逻辑回归的benchmark展现出来的结果相同。模型都存在一定的过拟合情况，其中GBDT在测试数据上表现出来的泛化能力有很大的问题，需要较为激进的调整。

In [331]:
fusion_estimators(clf)

train_score:1.0 
 cv_mean:0.9777390631049168 
 test_score:0.9805555555555555


很明显，通过集成模型的表现有了显著提升，交叉验证分数（97.77%）比任何单一学习器都高，且在测试集上的表现提升到了98.05%，可见当前数据下模型融合的效果十分显著。

||benchmark|基础融合|
|:-:|:-:|:-:|
|5折交叉验证|0.9666|<font color="green">**0.9777(↑)**</font>|
|测试集结果|0.9527|<font color="green">**0.9805(↑)**</font>|

当然了，并不是任意评估器、任意数据上都能够看到如此一目了然的结果。如果你进行投票或平均融合之后，融合的结果反而没有单个算法好，那你可能是落入了投票法与平均法会失效的几大陷阱：

> 1. **评估器之间的学习能力/模型表现差异太大**。在融合中，一个表现很差的模型会拉低整个融合模型的上限，尤其是回归类算法，当一个模型的表现很差时，平均法得出的结果很难比最好的单一算法还好。因此我们必须要使用表现相似的模型进行融合。如果你的评估器中有拖后腿的模型，无论这个模型有多么先进，都应该立刻把它剔除融合模型。<br><br>
> 2. **评估器在类型上太相似**，比如、全是树模型、都是Boosting算法，或都是线性评估器等。如果评估器类别太相似，模型融合会发挥不出作用，这在直觉上其实很好理解：如果平均/投票的评估器都一致，那融合模型最终得出的结果也会与单个评估器一致。<br><br>
> 3. **对评估器进行了过于精密的调优**。一般来说，我们可能会认为，先对模型进行调优后再融合，能够进一步提升模型的表现。经过粗略调优的评估器融合确实能提升模型表现，但如果对评估器进行过于精密的调优，可能会让融合后的算法处于严重过拟合的状态。因此，一般我们不会在评估器上进行太精准的调优。

★

- **第二组分类器**：略微调参（非精细化调参），限制过拟合

模型融合是一个可能加剧过拟合的手段，因此我们必须保证每一个学习器本身的过拟合不严重，为此我们需要对模型进行抗过拟合的处理。需要注意的是，扛过拟合可能会削弱模型的预测效果，因此我们必须根据过拟合的情况、泛化能力的展现来进行选择。对于逻辑回归，我们需要缩小参数`C`，对随机森林我们选择`max_depth`，对GBDT我们则选择`max_features`。其中，森林的树深需要使用tree_模块来查看：

In [116]:
#查看随机森林中的深度
clf2.fit(Xtrain,Ytrain)
for i in clf2.estimators_[:100]:
    print(i.tree_.max_depth)

13
13
15
15
13
13
13
14
12
13
15
13
12
15
13
13
15
12
13
13
15
13
12
14
15
14
12
14
15
13
12
12
14
13
12
13
14
13
12
14
14
14
12
14
13
13
14
10
13
12
15
14
12
12
14
13
12
15
12
14
12
15
13
13
12
13
14
12
14
12
14
12
14
12
14
14
14
14
12
13
14
15
14
15
14
13
14
13
12
12
12
13
14
12
15
14
12
13
11
13


In [353]:
clf1 = LogiR(max_iter = 3000, C=0.1, random_state=1412,n_jobs=8) #这一组合可能说明我们的max_iter设置得太大了
clf2 = RFC(n_estimators= 100,max_depth=12,random_state=1412,n_jobs=8)
clf3 = GBC(n_estimators= 100,max_features="sqrt",random_state=1412)

estimators = [("Logistic Regression",clf1), ("RandomForest", clf2), ("GBDT",clf3)]
clf = VotingClassifier(estimators,voting="soft")

In [354]:
individual_estimators(estimators)

Logistic Regression 
 train_score:1.0 
 cv_mean:0.9666085946573751 
 test_score:0.9638888888888889 

RandomForest 
 train_score:1.0 
 cv_mean:0.972875532326752 
 test_score:0.9805555555555555 

GBDT 
 train_score:1.0 
 cv_mean:0.9777342237708091 
 test_score:0.9722222222222222 



针对逻辑回归的调整效果不是非常明显，交叉验证的分数并未得到明显提升，但测试集上的结果上升，说明模型的泛化能力变得更加稳定了。但C参数是很少小于0.5的，我们现在得出的结果可能说明模型的权重是非常非常小的数，所以我们必须给与很小的C才能够加强过拟合的影响。随机森林的过拟合变得更加严重了，这说明我们限制过拟合对模型带来的伤害大于对模型的帮助，因此随机森林的过拟合是失败的，应该取消这种过拟合限制。GBDT的过拟合调整是非常成功的，交叉验证分数上升的同时，测试集上的分数也得到了大幅提升。结论是，我们保留对逻辑回归和GBDT的调整，撤销对随机森林的调整：

In [356]:
clf1 = LogiR(max_iter = 3000, C=0.1, random_state=1412,n_jobs=8) #这一组合可能说明我们的max_iter设置得太大了
clf2 = RFC(n_estimators= 100,random_state=1412,n_jobs=8)
clf3 = GBC(n_estimators= 100,max_features="sqrt",random_state=1412)

estimators = [("Logistic Regression",clf1), ("RandomForest", clf2), ("GBDT",clf3)]
clf = VotingClassifier(estimators,voting="soft")

In [357]:
fusion_estimators(clf)

train_score:1.0 
 cv_mean:0.9777463221060783 
 test_score:0.9861111111111112


||benchmark|基础融合|抗过拟合|
|:-:|:-:|:-:|:-:|
|5折交叉验证|0.9666|<font color="green">**0.9777(↑)**</font>|<font color="gray">**0.9777(-)**</font>|
|测试集结果|0.9527|<font color="green">**0.9805(↑)**</font>|<font color="green">**0.9861(↑)**</font>|

可以看到，经过过拟合调整后，模型的交叉验证分数与测试集上的分数都上升了，不过交叉验证分数是轻微上升（从0.9777392上升至0.9777463），这轻微的上升可以忽略不计。测试集上的分数则有了0.06%的提升，是一个显著的进步。

### **5 构建多样性**

在各个模型的过拟合情况较好、且模型稳定的情况下，我们要从哪个角度入手开始提升融合模型的表现呢？虽然我们不了解投票法的数学细节，但我们十分了解Bagging算法，比如随机森林的数学细节——我在<font color="red">**《2022机器学习正课》**</font>当中为大家详细证明了，为什么随机森林的结果会好于单个算法，并且列出了随机森林的结果好于单个算法的关键条件：**评估器之间相互独立**。评估器之间的独立性越强，则模型从平均/投票当中获得的方差减少就越大，模型整体的泛化能力就越强。

无论是投票法还是平均法，都与Bagging算法有异曲同工之妙，因此我们相信“独立性”也有助于提升投票融合与平均融合的效果。在模型融合当中，独立性被称为“多样性”（diversity），**评估器之间的差别越大、彼此之间就越独立，因此评估器越多样，独立性就越强**。完全独立的评估器在现实中几乎不可能实现，因为不同的算法执行的是相同的预测任务，更何况大多数时候算法们都在相同的数据上训练，因此评估器不可能完全独立。但我们有以下关键的手段，用来让评估器变得更多样、让评估器之间相对独立：

> - **训练数据多样性**：完成多组有效的特征工程，使用不同的特征矩阵训练不同的模型。该方法一般能够得到很好的效果，但如何找出多组有效的特征工程是难题。<br><br>
> - **样本多样性**：使用相同特征矩阵，但每次训练时抽样出不同的样本子集进行训练。当数据量较小时，抽样样本可能导致模型效果急剧下降。<br><br>
> - **特征多样性**：使用相同特征矩阵，但每次训练时抽样出不同的特征子集进行训练。当特征量较小时，抽样特征可能导致模型效果急剧下降。<br><br>
> - **随机多样性/训练多样性**：使用相同的算法，但使用不同的随机数种子（会导致使用不同的特征、样本、起点）、或使用不同的损失函数、使用不同的不纯度下降量等。这一方法相当于是在使用Bagging集成。<br><br>
> - **算法多样性**：增加类型不同的算法，如集成、树、概率、线性模型相混合。但需要注意的是，模型的效果不能太糟糕，无论是投票还是平均法，如果模型效果太差，可能大幅度降低融合的结果。

增加多样性的操作或多或少都阻止了模型学习完整的数据，因此会削弱模型对数据的学习，可能降低模型的效果。因此我们使用多样性时，需要时刻关注着模型的结果。

#### 5.1 多种多样性混合

In [77]:
#逻辑回归没有增加多样性的选项
clf1 = LogiR(max_iter = 3000, C=0.1, random_state=1412,n_jobs=8)
#增加特征多样性与样本多样性
clf2 = RFC(n_estimators= 100,max_features="sqrt",max_samples=0.9, random_state=1412,n_jobs=8)
#特征多样性，稍微上调特征数量
clf3 = GBC(n_estimators= 100,max_features=16,random_state=1412) 

#增加算法多样性，新增决策树、KNN、贝叶斯
clf4 = DTC(max_depth=8,random_state=1412)
clf5 = KNNC(n_neighbors=10,n_jobs=8)
clf6 = GaussianNB()

#新增随机多样性，相同的算法更换随机数种子
clf7 = RFC(n_estimators= 100,max_features="sqrt",max_samples=0.9, random_state=4869,n_jobs=8)
clf8 = GBC(n_estimators= 100,max_features=16,random_state=4869)

estimators = [("Logistic Regression",clf1), ("RandomForest", clf2)
              , ("GBDT",clf3), ("Decision Tree", clf4), ("KNN",clf5) 
              , ("Bayes",clf6), ("RandomForest2", clf7), ("GBDT2", clf8)
             ]
clf = VotingClassifier(estimators,voting="soft")

In [78]:
individual_estimators(estimators)

Logistic Regression 
 train_score:1.0 
 cv_mean:0.9666085946573751 
 test_score:0.9638888888888889 

RandomForest 
 train_score:1.0 
 cv_mean:0.9735699767711964 
 test_score:0.9777777777777777 

GBDT 
 train_score:1.0 
 cv_mean:0.9777414827719705 
 test_score:0.9722222222222222 

Decision Tree 
 train_score:0.9509369962538313 
 cv_mean:0.8552821331784747 
 test_score:0.8527777777777777 

KNN 
 train_score:0.9832990502137966 
 cv_mean:0.9763429152148664 
 test_score:0.9833333333333333 

Bayes 
 train_score:0.8295060354940024 
 cv_mean:0.8086285327138987 
 test_score:0.7888888888888889 

RandomForest2 
 train_score:1.0 
 cv_mean:0.9749491869918699 
 test_score:0.9833333333333333 

GBDT2 
 train_score:1.0 
 cv_mean:0.9770446186604724 
 test_score:0.975 



In [79]:
fusion_estimators(clf)

train_score:1.0 
 cv_mean:0.9826098528842431 
 test_score:0.9805555555555555


#### 5.2 剔除表现不良的算法，继续观察

In [81]:
estimators = [("Logistic Regression",clf1), ("RandomForest", clf2)
              , ("GBDT",clf3), ("Decision Tree", clf4), ("KNN",clf5) 
              #, ("Bayes",clf6) 贝叶斯在训练集上的分数很低，这说明模型的学习能力不足
              , ("RandomForest2", clf7), ("GBDT2", clf8)
             ]
clf = VotingClassifier(estimators,voting="soft")

In [82]:
individual_estimators(estimators)

Logistic Regression 
 train_score:1.0 
 cv_mean:0.9666085946573751 
 test_score:0.9638888888888889 

RandomForest 
 train_score:1.0 
 cv_mean:0.9735699767711964 
 test_score:0.9777777777777777 

GBDT 
 train_score:1.0 
 cv_mean:0.9777414827719705 
 test_score:0.9722222222222222 

Decision Tree 
 train_score:0.9509369962538313 
 cv_mean:0.8552821331784747 
 test_score:0.8527777777777777 

KNN 
 train_score:0.9832990502137966 
 cv_mean:0.9763429152148664 
 test_score:0.9833333333333333 

RandomForest2 
 train_score:1.0 
 cv_mean:0.9749491869918699 
 test_score:0.9833333333333333 

GBDT2 
 train_score:1.0 
 cv_mean:0.9770446186604724 
 test_score:0.975 



In [83]:
fusion_estimators(clf)

train_score:1.0 
 cv_mean:0.9833067169957413 
 test_score:0.9833333333333333


#### 5.3 尝试精简多样性

In [90]:
estimators = [("Logistic Regression",clf1), ("RandomForest", clf2)
              , ("GBDT",clf3), ("Decision Tree", clf4), ("KNN",clf5) 
              #, ("Bayes",clf6), ("RandomForest2", clf7), ("GBDT2", clf8)
             ]
clf = VotingClassifier(estimators,voting="soft")

In [91]:
individual_estimators(estimators)

Logistic Regression 
 train_score:1.0 
 cv_mean:0.9666085946573751 
 test_score:0.9638888888888889 

RandomForest 
 train_score:1.0 
 cv_mean:0.9735699767711964 
 test_score:0.9777777777777777 

GBDT 
 train_score:1.0 
 cv_mean:0.9777414827719705 
 test_score:0.9722222222222222 

Decision Tree 
 train_score:0.9509369962538313 
 cv_mean:0.8552821331784747 
 test_score:0.8527777777777777 

KNN 
 train_score:0.9832990502137966 
 cv_mean:0.9763429152148664 
 test_score:0.9833333333333333 



In [92]:
fusion_estimators(clf)

train_score:1.0 
 cv_mean:0.9819129887727449 
 test_score:0.9888888888888889


||benchmark|基础融合|抗过拟合|复合多样性|精简多样性|
|:-:|:-:|:-:|:-:|:-:|:-:|
|5折交叉验证|0.9666|<font color="green">**0.9777(↑)**</font>|<font color="gray">**0.9777(-)**</font>|<font color="green">**0.98331(↑)**</font>|<font color="brown">**0.9819(↓)**</font>|
|测试集结果|0.9527|<font color="green">**0.9805(↑)**</font>|<font color="green">**0.9861(↑)**</font>|<font color="brown">**0.98333(↓)**</font>|<font color="green">**0.9889(↑)**</font>|

在增加算法多样性的过程中，我们尝试了多种算法组合。我们发现朴素贝叶斯对算法的伤害大于贡献，只要将朴素贝叶斯包括在融合算法内，算法的表现就持续停留在97%左右，无法继续上升。因此我们删除了朴素贝叶斯算法。同时，包含随机多样性的算法组合在交叉验证与测试集结果上的表现高度一致，其中测试集结果略有降低、交叉验证结果提升了不少，这是一组可以使用的结果。但考虑到运算的效率，我们尽量不增加运算缓慢的集成算法，因此我们又将随机多样性删除，观察只包含5个算法的融合模型，最终交叉验证上的结果略有降低、测试集结果突破新高，这也是一组可以使用的结果。从模型的稳定性来考虑，还是包含随机多样性的组合更好，但为了更快的运算速率，我们可以使用精简多样性继续往下计算。至此，我们的融合模型就构建完毕了，接下来我们来调整融合模型。

### **6. 分类器加权**

对分类器加权是一个常见的操作，但如何选择权重却是整个模型融合过程中最令人头疼的问题——头疼主要在于，没有可以完全依赖的理论基础或数学公式去进行权重推导（或者说推导出来的权重基于过多假设、无法使用），同时费了很大精力求解出的权重可能对模型的效果完全没有影响。因此，在融合中加权是一个不经济的选项。但一般来说，我们还是会尝试几组权重来探索一下，模型是否还有提升的空间。

在机器学习算法中，只有一类算法对于评估器权重有自己的见解，那就是Boosting算法。在大部分Boosting集成过程中，我们会对每一个弱评估器求解其权重，并让权重作为迭代的一部分构建模型。那我们通常如何决定模型权重呢？在AdaBoost和XGBoost当中，我们都设立了用于衡量单个弱评估器置信度的某个指标，如果一个弱评估器的置信度越高，我们给与这个评估器的权重就越大。在Boosting算法中，置信度往往使用损失函数或损失函数的某种变体进行衡量，如果损失函数越大，则说明评估器的置信度越低，反之，则说明评估器的置信度越高。我们可以沿用这个思路，在模型融合中，我们会考虑的第一组权重，就是模型实际评估结果之间的比例。

如果模型评估指标是准确率这样的正向指标，则直接使用准确率作为权重。如果模型评估指标是MSE这样的负向指标，则使用1-指标或负指标作为权重。这样做的风险在于，模型可能陷入严重过拟合，但值得一试：

- 第一种选项：使用各个模型交叉验证结果本身作为权重，有过拟合风险

In [98]:
estimators = [("Logistic Regression",clf1), ("RandomForest", clf2)
              , ("GBDT",clf3), ("Decision Tree", clf4), ("KNN",clf5)]

In [99]:
clf_weighted = VotingClassifier(estimators,voting="soft",weights=[0.96660,0.97357,0.97774,0.85528,0.97634])

In [97]:
fusion_estimators(clf_weighted)

train_score:1.0 
 cv_mean:0.9826098528842431 
 test_score:0.9888888888888889


可以看到，模型在交叉验证上的效果提升了！模型没有陷入过拟合，这是一个很好的消息。接下来，我们可以尝试使用更粗糙的权重来提升模型表现。一般来说，这个操作可以一定程度上缓解模型的过拟合：

- 第二种选项：稍微降低权重精度，或许可以一定程度上抵消过拟合

In [20]:
clf_weighted = VotingClassifier(estimators,voting="soft",weights=[0.95,0.95,0.95,0.85,0.98])

In [21]:
fusion_estimators(clf_weighted)

train_score:1.0 
 cv_mean:0.9826098528842431 
 test_score:0.9888888888888889


可以看到，模型的表现并无变化，这说明对当前数据来说，精确的权重与粗略的权重差异不大。接下来，我们可以尝试调整模型中效果最好、或效果最差的算法的权重。一般，可以尝试加大效果好的算法的权重，减小效果差的算法的权重：

- 第三种选项：加大效果好的算法的权重，减小效果差的算法的权重

In [100]:
clf_weighted = VotingClassifier(estimators,voting="soft",weights=[0.95,0.95,0.95,0.85,1.2]) #增大

In [101]:
fusion_estimators(clf_weighted)

train_score:1.0 
 cv_mean:0.9826098528842431 
 test_score:0.9861111111111112


不难发现，过拟合开始发生了，测试集上的结果开始降低。因此我们选择不加大效果好的算法的权重。

In [103]:
clf_weighted = VotingClassifier(estimators,voting="soft",weights=[0.95,0.95,0.95,0.3,0.98]) #减小

In [104]:
fusion_estimators(clf_weighted)

train_score:1.0 
 cv_mean:0.9833067169957413 
 test_score:0.9888888888888889


模型达到了目前为止，在5折交叉验证及测试分数上的最高值。现在我们给与决策树的权重非常小，说明现在决策树只是在提供多样性，在实际预测方面做出的贡献较少。提供多样性可以让模型的泛化能力增强，让实际预测方面的贡献变小又可以降低决策树本身较低的预测分数带来的影响。至此，我们就得到了一个很好的融合结果。

||benchmark|基础融合|抗过拟合|复合多样性|精简多样性|权重调整|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|5折交叉验证|0.9666|<font color="green">**0.9777(↑)**</font>|<font color="gray">**0.9777(-)**</font>|<font color="green">**0.98331(↑)**</font>|<font color="brown">**0.9819(↓)**</font>|**0.9833**|
|测试集结果|0.9527|<font color="green">**0.9805(↑)**</font>|<font color="green">**0.9861(↑)**</font>|<font color="brown">**0.98333(↓)**</font>|<font color="green">**0.9889(↑)**</font>|**0.9889**|

In [ ]:
#版本确认
import re
import numpy as np
import pandas as pd
import matplotlib as mlp
import matplotlib.pyplot as plt
import sklearn
import xgboost as xgb

In [ ]:
for package in [sklearn,mlp,np,pd,xgb]:
    print(re.findall("([^']*)",str(package))[2],package.__version__)

sklearn 1.0.1
matplotlib 3.4.3
numpy 1.21.4
pandas 1.3.4
xgboost 1.5.1


In [ ]:
#pip install --upgrade scikit-learn
#conda update scikit-learn

#使用清华源配置xgboost
#!pip install xgboost -i https://pypi.tuna.tsinghua.edu.cn/simple

### **新年新班型，内容大更新**
- 十周高效充电，决胜金三银四
- 新增业务分析方法论
- 新增贝叶斯优化器和AutoML
- 新增CV医学影像识别
...

**更多实战案例、更多前沿技术**<br>
**内容迭代【超过30%】**<br>

<font color="Red" size=5>**融合直播课正式完结，在正课中将会包括：**</font><br>
- <font color="Red" size=4.5>**xgboost, lgbm等复杂集成算法上的融合(手写融合)**</font><br><br>
- <font color="Red" size=4.5>**基于Kaggle数据的融合调优实战**</font><br><br>
- <font color="Red" size=4.5>**gbdt+lr原理详解 + CTR融合实战**</font><br><br>
- <font color="Red" size=4.5>**如果融合无效应该怎么办？**</font>

<font color="green" size=4.5>**现在下单，可获新年活动优惠+直播特惠>>>**</font><br><br>
<font color="green" size=4.5>**微信扫码加小可爱，回复“ML”即可获得特惠>>>**</font>

# 一 堆叠法Stacking

## 1 堆叠法的基本思想

堆叠法Stacking是近年来模型融合领域最为热门的方法，它不仅是竞赛冠军队最常采用的融合方法之一，也是工业中实际落地人工智能时会考虑的方案之一。作为强学习器的融合方法，Stacking集**模型效果好、可解释性强、适用复杂数据**三大优点于一身，属于融合领域最为实用的先驱方法。在Stacking的众多应用当中，CTR（广告点击率预测）中实用的GBDT+LR堆叠尤为著名。因此在《2022机器学习实战》正课当中，讲解完毕通用的stacking手段之后，我会给大家详细讲解GBDT+LR在CTR中的用法，并完成一个CTR实战。

Stacking究竟是怎样一种算法呢？它的核心思想其实非常简单——首先，如下图所示，Stacking结构中有两层算法串联，第一层叫做level 0，第二层叫做level 1，level 0里面可能包含1个或多个强学习器，而level 1只能包含一个学习器。在训练中，数据会先被输入level 0进行训练，训练完毕后，level 0中的每个算法会输出相应的预测结果。我们将这些预测结果拼凑成新特征矩阵，再输入level 1的算法进行训练。融合模型最终输出的预测结果就是level 1的学习器输出的结果。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021MachineLearning/Ensembles/Fusion/10.png)

在这个过程中，level 0输出的预测结果一般如下排布：

||学习器1|学习器2|...|学习器n|
|:-:|:-:|:-:|:-:|:-:|
|样本1|xxx|xxx|...|xxx|
|样本2|xxx|xxx|...|xxx|
|样本3|xxx|xxx|...|xxx|
|...|...|...|...|...|
|样本m|xxx|xxx|...|xxx|

第一列就是学习器1在全部样本上输出的结果，第二列就是学习器2在全部样本上输出的结果，以此类推。

同时，level 0上训练的多个强学习器被称为基学习器(base-model)，也叫做个体学习器。在level1上训练的学习器叫元学习器（meta-model）。根据行业惯例，**level 0上的学习器是复杂度高、学习能力强的学习器**，例如集成算法、支持向量机，而**level 1上的学习器是可解释性强、较为简单的学习器**，如决策树、线性回归、逻辑回归等。有这样的要求是因为level 0上的算法们的职责是找出原始数据与标签的关系、即建立原始数据与标签之间的假设，因此需要强大的学习能力。但level 1上的算法的职责是融合个体学习器做出的假设、并最终输出融合模型的结果，相当于在寻找“最佳融合规则”，而非直接建立原始数据与标签之间的假设。

说到这里，不知道你是否有注意到，**Stacking的本质是让算法找出融合规则**。虽然大部分人可能从未接触过类似于Stacking算法的串联结构，但事实上Stacking的流程与投票法、均值法完全一致：

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021MachineLearning/Ensembles/Fusion/11.png)

在投票法中，我们用投票方式融合强学习器的结果，在均值法中，我们用求均值方式融合强学习器的结果，在Stacking堆叠法中，我们用算法融合强学习器的结果。当level 1上的算法是线性回归时，其实我们就是在求解所有强学习器结果的加权求和，而训练线性回归的过程，就是找加权求和的权重的过程。同样的，当level 1上的算法是逻辑回归的时候，其实我们就是在求解所有强学习器结果的加权求和，再在求和基础上套上sigmoid函数。训练逻辑回归的过程，也就是找加权求和的权重的过程。其他任意简单的算法同理。

虽然对大多数算法来说，我们难以找出类似“加权求和”这样一目了然的名字来概括算法找出的融合规则，但本质上，level 1的算法只是在学习如何将level 0上输出的结果更好地结合起来，所以**Stacking是通过训练学习器来融合学习器结果的方法**。这一方法的根本优势在于，我们可以让level 1上的元学习器向着损失函数最小化的方向训练，而其他融合方法只能保证融合后的结果有一定的提升。因此Stacking是比Voting和Averaging更有效的方法。在实际应用时，Stacking也常常表现出胜过投票或均值法的结果。

当我们了解了Stacking的本质之后，很多实现过程中的细节问题就迎刃而解了，比如：

- **要不要对融合的算法进行精密的调参？**<br>
> 个体学习器粗调，元学习器精调，如果不过拟合的话，可以两类学习器都精调。理论上来说，算法输出结果越接近真实标签越好，但个体学习器精调后再融合，很容易过拟合。<br>

- **个体学习器算法要怎样选择才能最大化stacking的效果？**<br>
> 与投票、平均的状况一致，控制过拟合、增加多样性、注意算法整体的运算时间。<br>

- **个体学习器可以是逻辑回归、决策树这种复杂度较低的算法吗？元学习器可以是xgboost这种复杂度很高的算法吗？**<br>
> 都可以，一切以模型效果为准。对level 0而言，当增加弱学习器来增加模型多样性、且弱学习器的效果比较好时，可以保留这些算法。对level 1而言，只要不过拟合，可以使用任何算法。个人推荐，在分类的时候可以使用复杂度较高的算法，对回归最好还是使用简单的算法。<br>

- **level 0和level 1的算法可不可以使用不同的损失函数？**<br>
> 可以，因为不同的损失函数衡量的其实是类似的差异：即真实值与预测值之间的差异。不过不同的损失对于差异的敏感性不同，如果可能的话建议使用相似的损失函数。<br>

- **level 0和level 1的算法可不可以使用不同的评估指标？**<br>
> 个人建议**level 0与level 1上的算法必须使用相同的模型评估指标**。虽然Stacking中串联了两组算法，但这两组算法的训练却是完全分离的。在深度学习当中，我们也有类似的强大算法串联弱小算法的结构，例如，卷积神经网络就是由强大的卷积层与弱小的线性层串联，卷积层的主要职责是找出特征与标签之间的假设，而线性层的主要职责是整合假设、进行输出。但在深度学习中，一个网络上所有层的训练是同时进行的，每次降低损失函数时需要更新整个网络上的权重。但在Stacking当中，level 1上的算法在调整权重时，完全不影响level 0的结果，因此为了保证两组算法最终融合后能够得到我们想要的结果，在训练时一定要以唯一评估指标为基准进行训练。<br>

## 2 在sklearn中实现stacking

相信到这里，你已经对Stacking算法的本质有足够深的认知了。在sklearn当中，我们使用如下两个类来实现stacking：

*class* `sklearn.ensemble.StackingClassifier`(estimators, final_estimator=None, *, cv=None, stack_method='auto', n_jobs=None, passthrough=False, verbose=0)<br>

*class* `sklearn.ensemble.StackingRegressor`(estimators, final_estimator=None, *, cv=None, n_jobs=None, passthrough=False, verbose=0)<br>

|参数|说明|
|:-:|-|
|estimators|个体评估器的列表。在sklearn中，只使用单一评估器作为个体评估器时，<br>模型可以运行，但效果往往不太好。|
|final_estimator|元学习器，只能有一个评估器。当融合模型执行分类任务时，元学习器一定是分类算法，<br>当融合模型执行回归任务时，元学习器一定是回归算法。|
|cv|用于指定交叉验证的具体类型、折数等细节。<br>可以执行简单的K折交叉验证，也可以输入sklearn中交叉验证类。|
|stack_method|只有分类器才有的参数，表示个体学习器输出的具体测试结果。|
|passthrough|在训练元学习器时，是否加入原始数据作为特征矩阵。|
|n_jobs, verbose|线程数与监控参数。|

在sklearn当中，只要输入`estimators`和`final_estimator`，就可以执行stacking了。我们可以沿用在投票法中使用过的个体学习器组合，并使用随机森林作为元学习器来完成stacking：

- **工具库 & 数据**

In [ ]:
#常用工具库
import re
import numpy as np
import pandas as pd
import matplotlib as mlp
import matplotlib.pyplot as plt
import time

#算法辅助 & 数据
import sklearn
from sklearn.model_selection import KFold, cross_validate
from sklearn.datasets import load_digits #分类 - 手写数字数据集
from sklearn.datasets import load_iris
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

#算法（单一学习器）
from sklearn.neighbors import KNeighborsClassifier as KNNC
from sklearn.neighbors import KNeighborsRegressor as KNNR
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import LogisticRegression as LogiR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import GradientBoostingRegressor as GBR
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb

#融合模型
from sklearn.ensemble import StackingClassifier

In [ ]:
data = load_digits()
X = data.data
y = data.target

In [ ]:
X.shape

(1797, 64)

In [ ]:
np.unique(y) #10分类

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,y,test_size=0.2,random_state=1412)

- **定义交叉验证函数**

In [ ]:
def fusion_estimators(clf):
    """
    对融合模型做交叉验证，对融合模型的表现进行评估
    """
    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    results = cross_validate(clf,Xtrain,Ytrain
                             ,cv = cv
                             ,scoring = "accuracy"
                             ,n_jobs = -1
                             ,return_train_score = True
                             ,verbose=False)
    test = clf.fit(Xtrain,Ytrain).score(Xtest,Ytest)
    print("train_score:{}".format(results["train_score"].mean())
          ,"\n cv_mean:{}".format(results["test_score"].mean())
          ,"\n test_score:{}".format(test)
         )

In [ ]:
def individual_estimators(estimators):
    """
    对模型融合中每个评估器做交叉验证，对单一评估器的表现进行评估
    """
    for estimator in estimators:
        cv = KFold(n_splits=5,shuffle=True,random_state=1412)
        results = cross_validate(estimator[1],Xtrain,Ytrain
                             ,cv = cv
                             ,scoring = "accuracy"
                             ,n_jobs = -1
                             ,return_train_score = True
                             ,verbose=False)
        test = estimator[1].fit(Xtrain,Ytrain).score(Xtest,Ytest)
        print(estimator[0]
          ,"\n train_score:{}".format(results["train_score"].mean())
          ,"\n cv_mean:{}".format(results["test_score"].mean())
          ,"\n test_score:{}".format(test)
          ,"\n")

- **个体学习器与元学习器的定义**

我们之前在讲解Voting投票法的时候已经对如何定义个体学习器做出了详细的解释，也做了很多努力找出下面的7个模型。在这里，我们就沿用Voting当中选出的7个模型：

In [ ]:
#逻辑回归没有增加多样性的选项
clf1 = LogiR(max_iter = 3000, C=0.1, random_state=1412,n_jobs=8)
#增加特征多样性与样本多样性
clf2 = RFC(n_estimators= 100,max_features="sqrt",max_samples=0.9, random_state=1412,n_jobs=8)
#特征多样性，稍微上调特征数量
clf3 = GBC(n_estimators= 100,max_features=16,random_state=1412) 

#增加算法多样性，新增决策树与KNN
clf4 = DTC(max_depth=8,random_state=1412)
clf5 = KNNC(n_neighbors=10,n_jobs=8)
clf6 = GaussianNB()

#新增随机多样性，相同的算法更换随机数种子
clf7 = RFC(n_estimators= 100,max_features="sqrt",max_samples=0.9, random_state=4869,n_jobs=8)
clf8 = GBC(n_estimators= 100,max_features=16,random_state=4869)

estimators = [("Logistic Regression",clf1), ("RandomForest", clf2)
              , ("GBDT",clf3), ("Decision Tree", clf4), ("KNN",clf5) 
              #, ("Bayes",clf6)
              , ("RandomForest2", clf7), ("GBDT2", clf8)
             ]

- **导入sklearn进行建模**

In [ ]:
#选择单个评估器中分数最高的随机森林作为元学习器
#也可以尝试其他更简单的学习器
final_estimator = RFC(n_estimators=100
                      , min_impurity_decrease=0.0025
                      , random_state= 420, n_jobs=8)
clf = StackingClassifier(estimators=estimators #level0的7个体学习器
                         ,final_estimator=final_estimator #level 1的元学习器
                         ,n_jobs=8)

In [ ]:
fusion_estimators(clf) #没有过拟合限制

train_score:1.0 
 cv_mean:0.9812112853271389 
 test_score:0.9861111111111112


In [ ]:
fusion_estimators(clf) #精调过拟合

train_score:1.0 
 cv_mean:0.9812185443283005 
 test_score:0.9888888888888889


||benchmark|投票法|堆叠法|
|:-:|:-:|:-:|:-:|
|5折交叉验证|0.9666|0.9833|<font color="brown">**0.9812(↓)**</font>|
|测试集结果|0.9527|0.9889|<font color="green">**0.9889(-)**</font>|

可以看到，stacking在测试集上的分数与投票法Voting持平，但在5折交叉验证分数上却没有投票法高。这可能是由于现在我们训练的数据较为简单，但数据学习难度较大时，stacking的优势就会慢慢显现出来。当然，我们现在使用的元学习器几乎是默认参数，我们可以针对元学习器使用贝叶斯优化进行精妙的调参，然后再进行对比，堆叠法的效果可能超越投票法。

## 3 元学习器的特征矩阵

### 3.1 元学习器特征矩阵的2个问题

在Stacking过程中，个体学习器会原始数据上训练、预测，再把预测结果排布成新特征矩阵，放入元学习器进行学习。其中，个体学习器的预测结果、即元学习器需要训练的矩阵一般如下排布：

||学习器1|学习器2|...|学习器n|
|:-:|:-:|:-:|:-:|:-:|
|样本1|xxx|xxx|...|xxx|
|样本2|xxx|xxx|...|xxx|
|样本3|xxx|xxx|...|xxx|
|...|...|...|...|...|
|样本m|xxx|xxx|...|xxx|

根据我们对机器学习以及模型融合的理解，不难发现以下两个问题：

- **首先，元学习器的特征矩阵中的特征一定很少**<br>

> 1个个体学习器只能输出1组预测结果，我们对这些预测结果进行排列，新特征矩阵中的特征数就等于个体学习器的个数。一般融合模型中个体学习器最多有20-30个，也就是说元学习器的特征矩阵中最多也就20-30个特征。这个特征量对于工业、竞赛中的机器学习算法来说是远远不够的。

- **其次，元学习器的特征矩阵中样本量也不太多**<br>

> 个体学习器的职责是找到原始数据与标签之间的假设，为了验证这个假设是否准确，我们需要查看的是个体学习器的泛化能力。只有当个体学习器的泛化能力较强时，我们才能安心的将个体学习器输出的预测结果放入元学习器中进行融合。<br><br>
> 然而。在我们训练stacking模型时，我们一定是将原始数据集分为训练集、验证集和测试集三部分——<br><br>
> 其中**测试集**是用于检测整个融合模型的效果的，因此在训练过程中不能使用。<br><br>
> 而**训练集**用于训练个体学习器，属于已经完全透露给个体学习器的内容，如果在训练集上进行预测，那预测结果是“偏高”的、无法代表个体学习器的泛化能力。<br><br>
> 因此最后剩下能够用来预测、还能代表个体学习器真实学习水平的，就只剩下很小的**验证集**了。一般验证集最多只占整个数据集的30%-40%，这意味着元学习器所使用的特征矩阵里的样本量最多也就是原始数据的40%。

无怪在行业惯例当中，元学习器需要是一个复杂度较低的算法，因为元学习器的特征矩阵在特征量、样本量上都远远小于工业机器学习所要求的标准。为了解决这两个问题，在Stacking方法当中存在多种解决方案，而这些解决方案可以通过sklearn中的stacking类实现。

*class* `sklearn.ensemble.StackingClassifier`(estimators, final_estimator=None, *, cv=None, stack_method='auto', n_jobs=None, passthrough=False, verbose=0)<br>

*class* `sklearn.ensemble.StackingRegressor`(estimators, final_estimator=None, *, cv=None, n_jobs=None, passthrough=False, verbose=0)<br>

|参数|说明|
|:-:|-|
|estimators|个体评估器的列表。在sklearn中，只使用单一评估器作为个体评估器时，<br>模型可以运行，但效果往往不太好。|
|final_estimator|元学习器，只能有一个评估器。当融合模型执行分类任务时，元学习器一定是分类算法，<br>当融合模型执行回归任务时，元学习器一定是回归算法。|
|cv|用于指定交叉验证的具体类型、折数等细节。<br>可以执行简单的K折交叉验证，也可以输入sklearn中交叉验证类。|
|stack_method|只有分类器才有的参数，表示个体学习器输出的具体测试结果。|
|passthrough|在训练元学习器时，是否加入原始数据作为特征矩阵。|
|n_jobs, verbose|线程数与监控参数。|

### 3.2 样本量太少的解决方案：交叉验证

- **参数`cv`，在stacking中执行交叉验证**

在stacking方法被提出的原始论文当中，原作者自然也意识到了元学习器的特征矩阵样本量太少这个问题，因此提出了**在stacking流程内部使用交叉验证来扩充元学习器特征矩阵**的想法，即在内部对每个个体学习器做交叉验证，但并不用这个交叉验证的结果来验证泛化能力，而是直接把交叉验证当成了生产数据的工具。

具体的来看，在stacking过程中，我们是这样执行交叉验证的——

对任意个体学习器来说，假设我们执行5折交叉验证，我们会将训练数据分成5份，并按照4份训练、1份验证的方式总共建立5个模型，训练5次：

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021MachineLearning/Ensembles/Fusion/12.png)

在交叉验证过程中，每次验证集中的数据都是没有被放入模型进行训练的，因此这些验证集上的预测结果都可以衡量模型的泛化能力。

一般来说，交叉验证的最终输出是5个验证集上的分数，但计算分数之前我们一定是在5个验证集上分别进行预测，并输出了结果。所以我们可以在交叉验证中**建立5个模型，轮流得到5个模型输出的预测结果，而这5个预测结果刚好对应全数据集中分割的5个子集**。这是说，我们完成交叉验证的同时，也对原始数据中全部的数据完成了预测。现在，只要将5个子集的预测结果**纵向堆叠**，就可以得到一个和原始数据中的样本一一对应的预测结果。这种纵向堆叠正像我们在海滩上堆石子(stacking)一样，这也是“堆叠法”这个名字的由来。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021MachineLearning/Ensembles/Fusion/14.png)

用这样的方法来进行预测，可以让**任意个体学习器输出的预测值数量 = 样本量**，如此，元学习器的特征矩阵的行数也就等于原始数据的样本量了：

||学习器1|学习器2|...|学习器n|
|:-:|:-:|:-:|:-:|:-:|
|样本1|xxx|xxx|...|xxx|
|样本2|xxx|xxx|...|xxx|
|样本3|xxx|xxx|...|xxx|
|...|...|...|...|...|
|样本m|xxx|xxx|...|xxx|

在stacking过程中，这个交叉验证流程是一定会发生的，不属于我们可以人为干涉的范畴。不过，我们可以使用参数`cv`来决定具体要使用怎样的交叉验证，包括具体使用几折验证，是否考虑分类标签的分布等等。具体来说，参数`cv`中可以输入：

> 输入**None**，默认使用5折交叉验证<br><br>
> 输入sklearn中**任意交叉验证对象**<br><br>
> 输入**任意整数**，表示在Stratified K折验证中的折数。Stratified K折验证是会考虑标签中每个类别占比的交叉验证，如果选择Stratified K折交叉验证，那每次训练时交叉验证会保证**原始标签中的类别比例 = 训练标签的类别比例 = 验证标签的类别比例**。

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_009.png)

现在你知道Stacking是如何处理元学习器的特征矩阵样本太少的问题了。需要再次强调的是，内部交叉验证的并不是在验证泛化能力，而是一个生产数据的工具，因此交叉验证本身没有太多可以调整的地方。唯一值得一提的是，当交叉验证的折数较大时，模型的抗体过拟合能力会上升、同时学习能力会略有下降。当交叉验证的折数很小时，模型更容易过拟合。但如果数据量足够大，那使用过多的交叉验证折数并不会带来好处，反而只会让训练时间增加而已：

In [ ]:
estimators = [("Logistic Regression",clf1), ("RandomForest", clf2)
              , ("GBDT",clf3), ("Decision Tree", clf4), ("KNN",clf5) 
              #, ("Bayes",clf6)
              , ("RandomForest2", clf7), ("GBDT2", clf8)
             ]
final_estimator = RFC(n_estimators=100
                      , min_impurity_decrease=0.0025
                      , random_state= 420, n_jobs=8)

In [ ]:
def cvtest(cv):
    clf = StackingClassifier(estimators=estimators
                         ,final_estimator=final_estimator
                         , cv = cv
                         , n_jobs=8)
    start = time.time()
    clf.fit(Xtrain,Ytrain)
    print((time.time() - start)) #消耗时间
    print(clf.score(Xtrain,Ytrain)) #训练集上的结果
    print(clf.score(Xtest,Ytest)) #测试集上的结果

In [ ]:
cvtest(2) #非常少的验证次数

3.8339908123016357
1.0
0.9861111111111112


In [ ]:
cvtest(10) #普通的验证次数

13.57864761352539
1.0
0.9833333333333333


In [ ]:
cvtest(30) #很大的验证次数

39.74843621253967
1.0
0.9833333333333333


可以看到，随着cv中折数的上升，训练时间一定会上升，但是模型的表现却不一定。因此，选择5~10折交叉验证即可。同时，由于stacking当中自带交叉验证，又有元学习器这个算法，因此**堆叠法的运行速度是比投票法、均值法缓慢很多的**，这是stacking堆叠法不太人性化的地方。

### 3.3 特征太少的解决方案

- **参数`stack_method`，更换个体学习器输出的结果类型**

对于分类stacking来说，如果特征量太少，我们可以更换个体学习器输出的结果类型。具体来说，如果个体学习器输出的是具体类别（如[0,1,2]），那1个个体学习器的确只能输出一列预测结果。但如果把输出的结果类型更换成概率值、置信度等内容，输出结果的结构一下就可以从一列拓展到多列。

如果这个行为由参数`stack_method`控制，这是只有StackingClassifier才拥有的参数，它控制个体分类器具体的输出。`stack_method`里面可以输入四种字符串："auto", "predict_proba", "decision_function", "predict"，除了"auto"之外其他三个都是sklearn常见的接口。

In [ ]:
clf = LogiR(max_iter=3000, random_state=1412)

In [ ]:
clf = clf.fit(Xtrain,Ytrain)

In [ ]:
#predict_proba：输出概率值
clf.predict_proba(Xtrain)

array([[1.11319443e-16, 9.89024228e-16, 3.60767000e-21, ...,
        6.19650864e-16, 1.05063229e-17, 2.55116054e-13],
       [9.99999914e-01, 2.54891883e-20, 4.18412585e-13, ...,
        7.90066220e-12, 6.23126500e-10, 2.48485150e-11],
       [6.49037836e-11, 8.53171613e-08, 9.99958193e-01, ...,
        1.32963240e-15, 3.00724259e-05, 2.24500784e-17],
       ...,
       [2.86089528e-11, 1.41217666e-13, 9.81533656e-01, ...,
        5.70317183e-13, 5.74658333e-06, 3.46353786e-14],
       [4.08909180e-16, 1.13229614e-19, 1.94549576e-12, ...,
        4.15099225e-13, 2.78827639e-08, 9.73650086e-09],
       [5.33535502e-11, 1.56677615e-03, 2.20723889e-18, ...,
        4.83104216e-06, 3.16743744e-06, 9.98423722e-01]])

In [ ]:
#decision_function：每个样本点到分类超平面的距离，可以衡量置信度
#对于无法输出概率的算法，如SVM，我们通常使用decision_function来输出置信度
clf.decision_function(Xtrain)

array([[ -4.01299954,  -1.82868464, -14.35009661, ...,  -2.29624727,
         -6.37342621,   3.72407037],
       [ 23.41405676, -21.70197573,  -5.08825156, ...,  -2.15001769,
          2.21778528,  -1.00416628],
       [ -5.30159368,   1.8796313 ,  18.15647971, ..., -16.09735237,
          7.74461962, -20.17871606],
       ...,
       [ -3.67534951,  -8.98652206,  20.58331292, ...,  -7.59063183,
          8.53504682, -10.39194883],
       [ -7.61598125, -15.80781186,   0.85155307, ...,  -0.69320146,
         10.42180022,   9.36967331],
       [ -6.39460948,  10.80073594, -23.39531819, ...,   5.01902278,
          4.59688345,  17.2578936 ]])

In [ ]:
#predict：输出具体的预测标签
clf.predict(Xtrain)

array([5, 0, 2, ..., 2, 3, 9])

对参数`stack_method`有：
> 输入"auto"，sklearn会在每个个体学习器上按照"predict_proba", "decision_function", "predict"的顺序，分别尝试学习器可以使用哪个接口进行输出。即，如果一个算法可以使用predict_proba接口，那就不再尝试后面两个接口，如果无法使用predict_proba，就尝试能否使用decision_function。<br><br>
> 输入三大接口中的任意一个接口名，则默认全部个体学习器都按照这一接口进行输出。然而，如果遇见某个算法无法按照选定的接口进行输出，stacking就会报错。

因此，我们一般都默认让`stack_method`保持为"auto"。从上面的我们在逻辑回归上尝试的三个接口结果来看，很明显，当我们把输出的结果类型更换成概率值、置信度等内容，输出结果的结构一下就可以从一列拓展到多列。

- **predict_proba**
> 对二分类，输出样本的真实标签1的概率，一列<br>
> 对n分类，输出样本的真实标签为[0,1,2,3...n]的概率，一共n列

- **decision_function**
> 对二分类，输出样本的真实标签为1的置信度，一列<br>
> 对n分类，输出样本的真实标签为[0,1,2,3...n]的置信度，一共n列

- **predict**
> 对任意分类形式，输出算法在样本上的预测标签，一列

在实践当中，我们会发现输出概率/置信度的效果比直接输出预测标签的效果好很多，既可以向元学习器提供更多的特征、还可以向元学习器提供个体学习器的置信度。我们在投票法中发现使用概率的“软投票”比使用标签类被的“硬投票”更有效，也是因为考虑了置信度。

- **参数`passthrough`，将原始特征矩阵加入新特征矩阵**

对于分类算法，我们可以使用`stack_method`，但是对于回归类算法，我们没有这么多可以选择的接口。回归类算法的输出永远就只有一列连续值，因而我们可以考虑将原始特征矩阵加入个体学习器的预测值，构成新特征矩阵。这样的话，元学习器所使用的特征也不会过于少了。当然，这个操作有较高的过拟合风险，因此当特征过于少、且stacking算法的效果的确不太好的时候，我们才会考虑这个方案。

控制是否将原始数据加入特征矩阵的参数是`passthrough`，我们可以在该参数中输入布尔值。当设置为False时，表示不将原始特征矩阵加入个体学习器的预测值，设置为True时，则将原始特征矩阵加入个体学习器的预测值、构成大特征矩阵。

- 接口`transform`与属性`stack_method_`

In [ ]:
estimators = [("Logistic Regression",clf1), ("RandomForest", clf2)
              , ("GBDT",clf3), ("Decision Tree", clf4), ("KNN",clf5) 
              #, ("Bayes",clf6)
              , ("RandomForest2", clf7), ("GBDT2", clf8)
             ]

In [ ]:
final_estimator = RFC(n_estimators=100
                      , min_impurity_decrease=0.0025
                      , random_state= 420, n_jobs=8)
clf = StackingClassifier(estimators=estimators
                         ,final_estimator=final_estimator
                         ,stack_method = "auto"
                         ,n_jobs=8)

In [ ]:
clf = clf.fit(Xtrain,Ytrain)

当我们训练完毕stacking算法后，可以使用接口`transform`来查看当前**元学习器所使用的训练特征矩阵的结构**：

In [ ]:
clf.transform(Xtrain).shape

(1437, 70)

如之前所说，这个特征矩阵的行数就等于训练的样本量：

In [ ]:
Xtrain.shape[0]

1437

不过你能判断为什么这里有70列吗？因为我们有7个个体学习器，而现在数据是10分类的数据，因此每个个体学习器都输出了类别[0,1,2,3,4,5,6,7,8,9]所对应的概率，因此总共产出了70列数据：

In [ ]:
pd.DataFrame(clf.transform(Xtrain)).head()

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,3.365514e-11,2.965868e-10,9.917224e-14,1.915620e-08,1.936013e-10,1.000000e+00,1.117622e-11,3.264172e-10,4.190639e-11,2.062120e-08,...,2.002212e-07,3.143108e-06,6.019771e-07,1.149528e-05,0.000002,0.999965,4.110135e-07,8.580429e-07,0.000004,0.000012
1,9.999838e-01,1.537041e-13,5.214720e-09,5.462467e-09,1.400443e-07,1.425684e-05,6.408482e-07,2.707380e-08,1.012484e-06,1.428500e-07,...,9.999684e-01,3.986468e-07,7.661002e-07,8.657756e-07,0.000001,0.000011,4.000087e-06,3.828487e-07,0.000003,0.000010
2,8.525936e-08,1.925708e-05,9.985917e-01,1.807765e-04,3.228044e-07,4.127797e-04,1.622290e-04,2.915310e-10,6.328172e-04,6.198298e-11,...,4.862091e-07,3.863066e-05,9.996182e-01,9.759533e-05,0.000011,0.000058,2.555649e-05,1.856421e-06,0.000124,0.000024
3,1.855973e-07,1.459770e-08,3.227448e-12,8.743667e-14,9.999641e-01,1.467078e-10,1.379078e-06,3.424530e-05,4.306753e-08,2.199567e-11,...,7.867807e-07,2.055728e-06,6.228280e-07,8.675724e-07,0.999765,0.000001,3.959476e-06,2.081655e-04,0.000010,0.000008
4,7.470809e-05,9.309219e-07,2.840082e-06,3.902514e-06,1.470382e-03,2.121347e-03,8.517297e-08,9.958360e-01,3.160166e-04,1.737619e-04,...,3.493104e-06,6.057915e-05,2.445526e-05,6.996183e-05,0.000096,0.000104,1.423883e-05,9.984240e-01,0.000714,0.000490


如果加入参数`passthrough`，特征矩阵的特征量会变得更大：

In [ ]:
clf = StackingClassifier(estimators=estimators
                         ,final_estimator=final_estimator
                         ,stack_method = "auto"
                         ,passthrough = True
                         ,n_jobs=8)

In [ ]:
clf = clf.fit(Xtrain,Ytrain)

In [ ]:
clf.transform(Xtrain).shape #这里就等于70 + 原始特征矩阵的特征数量64

(1437, 134)

使用属性`stack_method_`，我们可以查看现在每个个体学习器都使用了什么接口做为预测输出：

In [ ]:
clf.stack_method_ 

['predict_proba',
 'predict_proba',
 'predict_proba',
 'predict_proba',
 'predict_proba',
 'predict_proba',
 'predict_proba']

不难发现，7个个体学习器都使用了`predict_proba`的概率接口进行输出，这与我们选择的算法都是可以输出概率的算法有很大的关系。

## 4 Stacking融合的训练/测试流程

现在我们已经知道了stacking算法中所有关于训练的信息，我们可以梳理出如下训练流程：

- **stacking的训练**
> 1. 将数据分割为训练集、测试集，其中训练集上的样本为$M_{train}$，测试集上的样本量为$M_{test}$<br><br>
> 2. 将训练集输入level 0的个体学习器，分别在每个个体学习器上进行交叉验证。在每个个体学习器上，将所有交叉验证的验证结果纵向堆叠形成预测结果。假设预测结果为概率值，当融合模型执行回归或二分类任务时，该预测结果的结构为$(M_{train},1)$，当融合模型执行K分类任务时(K>2)，该预测结果的结构为$(M_{train},K)$<br><br>
> 3. 将所有个体学习器的预测结果横向拼接，形成新特征矩阵。假设共有N个个体学习器，则新特征矩阵的结构为$(M_{train}, N)$。如果是输出多分类的概率，那最终得出的新特征矩阵的结构为$(M_{train}, N*K)$<br><br>
> 4. 将新特征矩阵放入元学习器进行训练。

现在，你能梳理出Stacking的测试流程吗？

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021MachineLearning/Ensembles/Fusion/10.png)

不难发现，虽然训练的流程看起来比较流畅，但是测试却不知道从何做起，因为：

- 最终输出预测结果的是元学习器，因此直觉上来说测试数据集或许应该被输入到元学习器当中。然而，**元学习器是使用新特征矩阵进行预测的，新特征矩阵的结构与规律都与原始数据不同**，所以元学习器根本不可能接受从原始数据中分割出来的测试数据。因此正确的做法应该是让测试集输入level 0的个体学习器。

- 然而，这又存在问题了：level 0的个体学习器们在训练过程中做的是交叉验证，而**交叉验证只会输出验证结果，不会留下被训练的模型**。因此在level 0中没有可以用于预测的、已经训练完毕的模型。

为了解决这个矛盾在我们的训练流程中，存在着隐藏的步骤：

- **stacking的训练**
> 1. 将数据分割为训练集、测试集，其中训练集上的样本为$M_{train}$，测试集上的样本量为$M_{test}$<br><br>
> 2. 将训练集输入level 0的个体学习器，分别在每个个体学习器上进行交叉验证。在每个个体学习器上，将所有交叉验证的验证结果纵向堆叠形成预测结果。假设预测结果为概率值，当融合模型执行回归或二分类任务时，该预测结果的结构为$(M_{train},1)$，当融合模型执行K分类任务时(K>2)，该预测结果的结构为$(M_{train},K)$<br><br>
> 3.  <font color="orange">**隐藏步骤：使用全部训练数据对所有个体学习器进行训练，为测试做好准备。**</font><br><br>
> 4. 将所有个体学习器的预测结果横向拼接，形成新特征矩阵。假设共有N个个体学习器，则新特征矩阵的结构为$(M_{train}, N)$.<br><br>
> 5. 将新特征矩阵放入元学习器进行训练。

- **stacking的测试**
> 1. 将测试集输入level0的个体学习器，分别在每个个体学习器上预测出相应结果。假设测试结果为概率值，当融合模型执行回归或二分类任务时，该测试结果的结构为$(M_{test},1)$，当融合模型执行K分类任务时(K>2)，该测试结果的结构为$(M_{test},K)$<br><br>
> 2. 将所有个体学习器的预测结果横向拼接为新特征矩阵。假设共有N个个体学习器，则新特征矩阵的结构为$(M_{test}, N)$.<br><br>
> 3. 将新特征矩阵放入元学习器进行预测。

因此在stacking中，不仅要对个体学习器完成全部交叉验证，还需要在交叉验证结束后，重新使用训练数据来训练所有的模型。无怪Stacking融合的复杂度较高、并且运行缓慢了。

到现在，我们已经讲解完毕投票法和堆叠法了。在sklearn中，我们讲解了下面4个类：

|融合方法|类|
|:-:|:-:|
|投票法|ensemble.VotingClassifier|
|平均法|ensemble.VotingRegressor|
|堆叠法分类|ensemble.StackingClassifier|
|堆叠法回归|ensemble.StackingRegressor|

虽然这些类是模型融合方法，但我们可以像使用任意单一算法类一样任意地使用这些方法——我们可以很轻松地对这些类执行手动调参、交叉验证、网格搜索、贝叶斯优化、管道打包等操作，而无需担心代码的兼容问题。**但需要注意的是，sklearn中的融合工具只支持sklearn中的评估器，不支持xgb、lgbm的原生代码**。因此，如果我们想要对原生代码下的模型进行融合，必须自己手写融合过程。<font color="red">**在正课《2022机器学习实战》当中，我将会手写融合过程，带大家来融合超强的Boosting学习器们。**</font>

# 二 改进后的堆叠法：Blending

## 1 Blending的基本思想与流程

Blending融合是在Stacking融合的基础上改进过后的算法。在之前的课程中我们提到，堆叠法stacking在level1上使用算法，这可以令融合本身向着损失函数最小化的方向进行，同时stacking使用自带的内部交叉验证来生成数据，可以深度使用训练数据，让模型整体的效果更好。但在这些操作的背后，存在两个巨大的问题：

- **stacking融合需要巨大的计算量，需要的时间和算力成本较高**，以及

- **stacking融合在数据和算法上都过于复杂，因此融合模型过拟合的可能性太高**。

针对stacking存在的这两个问题，竞赛冠军队们持续探索，并且在实践过程中创造了多种改进的stacking方法。今天，多种stacking方法中较为有效的方法之一就是著名的Blending方法。Blending直译为“混合”，但它的核心思路其实与Stacking完全一致：使用两层算法串联，level0上存在多个强学习器，level1上有且只有一个元学习器，且level0上的强学习器负责拟合数据与真实标签之间的关系、并输出预测结果、组成新的特征矩阵，然后让level1上的元学习器在新的特征矩阵上学习并预测。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021MachineLearning/Ensembles/Fusion/10.png)

然而，与stacking不同的是，为了降低计算量、降低融合模型过拟合风险，Blending取消了K折交叉验证、并且大大地降低了元学习器所需要训练的数据量，其具体流程如下：

- **blending的训练**
> 1. 将数据分割为训练集、验证集与测试集，其中训练集上的样本为$M_{train}$，验证集上的样本为$M_v$，测试集上的样本量为$M_{test}$<br><br>
> 2. 将训练集输入level 0的个体学习器，分别在每个个体学习器上训练。训练完毕后，在验证集上进行验证，输出验证集上的预测结果。假设预测结果为概率值，当融合模型执行回归或二分类任务时，该预测结果的结构为$(M_v,1)$，当融合模型执行K分类任务时(K>2)，该预测结果的结构为$(M_v,K)$。此时此刻，所有个体学习器都被训练完毕了。<br><br>
> 3. 将所有个体学习器的验证结果横向拼接，形成新特征矩阵。假设共有N个个体学习器，则新特征矩阵的结构为$(M_v, N)$。<br><br>
> 4. 将新特征矩阵放入元学习器进行训练。

- **blending的测试**
> 1. 将测试集输入level0的个体学习器，分别在每个个体学习器上预测出相应结果。假设测试结果为概率值，当融合模型执行回归或二分类任务时，该测试结果的结构为$(M_{test},1)$，当融合模型执行K分类任务时(K>2)，该测试结果的结构为$(M_{test},K)$<br><br>
> 2. 将所有个体学习器的预测结果横向拼接为新特征矩阵。假设共有N个个体学习器，则新特征矩阵的结构为$(M_{test}, N)$.<br><br>
> 3. 将新特征矩阵放入元学习器进行预测。

在讲解Stacking时我们提到：在实际进行训练时，验证集肯定是远远小于训练集的，因此只使用一部分验证集进行预测的方法一定会让新特征矩阵的尺寸变得非常小。在大部分时候，这是一个劣势，但在数据量庞大、运算成本极高的场景下，只使用一部分验证集构建新特征矩阵，反而还能提升运算速度、降低运算成本、防止模型过度学习、并提升模型的抗过拟合能力。**很显然的，验证集越大，模型抗过拟合能力越强，同时学习能力越弱。**

在实际应用时，如果数据量较大、且stacking方法表现出过拟合程度很高，那换blending融合可以获得更好的结果。相对的，如果stacking算法没有表现出太强的过拟合，那换blending融合可能导致模型的学习能力极速下降。

有一种场景下，stacking和blending都无法发挥效用，即数据量很小、且stacking表现出强烈过拟合的情况。这种状况下，数据或许不适合模型融合场景，或者我们可以更换成规则更简单的融合方式，例如平均、投票等来查看模型的过拟合情况。在实际中，stacking的应用是远比blending广泛的。

截止到2022年1月下旬，sklearn还不支持Blending方法，因此要实现Blending我们只能自己动手丰衣足食了。幸运的是，Blending的训练流程相当简单，因此我们可以很容易实现它。

## 2 手动实现Blending算法

In [ ]:
def BlendingClassifier(X,y,estimators,final_estimator,test_size=0.2,vali_size=0.4):
    """
    该函数用于实现Blending分类融合
    X,y：整体数据集，会被分割为训练集、测试集、验证集三部分
    estimators: level0的个体学习器，输入格式形如sklearn中要求的[(名字，算法)，(名字，算法)...]    
    final_estimator：元学习器
    test_size：测试集占全数据集的比例
    vali_size：验证集站全数据集的比例
    
    """
    
    #第一步，分割数据集
    #1.分测试集
    #2.分训练和验证集，验证集占完整数据集的比例为0.4，因此占排除测试集之后的比例为0.4/(1-0.2)
    X_,Xtest,y_,Ytest = train_test_split(X,y,test_size=test_size,random_state=1412)
    Xtrain,Xvali,Ytrain,Yvali = train_test_split(X_,y_,test_size=vali_size/(1-test_size),random_state=1412)
    
    #训练
    #建立空dataframe用于保存个体学习器上的验证结果，即用于生成新特征矩阵
    #新建空列表用于保存训练完毕的个体学习器，以便在测试中使用
    NewX_vali = pd.DataFrame()
    trained_estimators = []
    #循环、训练每个个体学习器、并收集个体学习器在验证集上输出的概率
    for clf_id, clf in estimators:
        clf = clf.fit(Xtrain,Ytrain)
        val_predictions = pd.DataFrame(clf.predict_proba(Xvali))
        #保存结果，在循环中逐渐构筑新特征矩阵
        NewX_vali = pd.concat([NewX_vali,val_predictions],axis=1)
        trained_estimators.append((clf_id,clf))
    #元学习器在新特征矩阵上训练、并输出训练分数
    final_estimator = final_estimator.fit(NewX_vali,Yvali)
    train_score = final_estimator.score(NewX_vali,Yvali)
    
    #测试
    #建立空dataframe用于保存个体学习器上的预测结果，即用于生成新特征矩阵
    NewX_test = pd.DataFrame()
    #循环，在每个训练完毕的个体学习器上进行预测，并收集每个个体学习器上输出的概率
    for clf_id,clf in trained_estimators:
        test_prediction = pd.DataFrame(clf.predict_proba(Xtest))
        #保存结果，在循环中逐渐构筑特征矩阵
        NewX_test = pd.concat([NewX_test,test_prediction],axis=1)
    #元学习器在新特征矩阵上测试、并输出测试分数
    test_score = final_estimator.score(NewX_test,Ytest)
    
    #打印训练分数与测试分数
    print(train_score,test_score)

In [ ]:
#逻辑回归没有增加多样性的选项
clf1 = LogiR(max_iter = 3000, C=0.1, random_state=1412,n_jobs=8)
#增加特征多样性与样本多样性
clf2 = RFC(n_estimators= 100,max_features="sqrt",max_samples=0.9, random_state=1412,n_jobs=8)
#特征多样性，稍微上调特征数量
clf3 = GBC(n_estimators= 100,max_features=16,random_state=1412) 

#增加算法多样性，新增决策树与KNN
clf4 = DTC(max_depth=8,random_state=1412)
clf5 = KNNC(n_neighbors=10,n_jobs=8)
clf6 = GaussianNB()

#新增随机多样性，相同的算法更换随机数种子
clf7 = RFC(n_estimators= 100,max_features="sqrt",max_samples=0.9, random_state=4869,n_jobs=8)
clf8 = GBC(n_estimators= 100,max_features=16,random_state=4869)

estimators = [("Logistic Regression",clf1), ("RandomForest", clf2)
              , ("GBDT",clf3), ("Decision Tree", clf4), ("KNN",clf5) 
              #, ("Bayes",clf6)
              #, ("RandomForest2", clf7), ("GBDT2", clf8)
             ]

In [ ]:
final_estimator = RFC(n_estimators= 100
                      #, max_depth = 8
                      , min_impurity_decrease=0.0025
                      , random_state= 420, n_jobs=8)

In [ ]:
#很明显，过拟合程度比Stacking要轻，但是测试集的表现没有stacking强
BlendingClassifier(X,y,estimators,final_estimator)

0.9958275382475661 0.9694444444444444


In [ ]:
#验证比例越大，模型学习能力越弱 - 注意验证集比例上限0.8，因为有0.2是测试数据
BlendingClassifier(X,y,estimators,final_estimator,vali_size=0.7)

0.9833068362480127 0.9472222222222222


In [ ]:
#相信大家可以感觉到，blending的运行速度比stacking快了不止一个档次……
BlendingClassifier(X,y,estimators,final_estimator,vali_size=0.1)

1.0 0.9833333333333333


||benchmark|投票法|Stacking|Blending|
|:-:|:-:|:-:|:-:|:-:|
|5折交叉验证|0.9666|0.9833|<font color="brown">**0.9812(↓)**</font>|-|
|测试集结果|0.9527|0.9889|<font color="green">**0.9889(-)**</font>|<font color="brown">**0.9833(↓)**</font>|

从结果来看，投票法表现最稳定和优异，这与我们选择的数据集是较为简单的数据集有关，同时投票法也是我们调整最多、最到位的算法。在大型数据集上运行时，stacking和blending会展现出更多的优势。到这里我们的blending就讲解完毕了，在《2022机器学习实战》正式课程当中，我们将会更详细地讲解Blending在xgboost等复杂算法上的应用。